In [1]:
import xarray as xr
import numpy as np
import xesmf as xe
import matplotlib.pyplot as plt
from math import nan
import geocat.comp as gc

In [2]:
outpath="/glade/scratch/islas/ERA5interp/"

### Set up parameters

In [3]:
R=287.053
g=9.81
dtdzst = -6.5e-3

### Read in the ERA5 data

In [4]:
era5path="/glade/collections/rda/data/ds633.6/e5.oper.an.ml/202106/"
t_era5 = xr.open_dataset(era5path+"e5.oper.an.ml.0_5_0_0_0_t.regn320sc.2021060100_2021060105.nc")
u_era5 = xr.open_dataset(era5path+"e5.oper.an.ml.0_5_0_2_2_u.regn320uv.2021060100_2021060105.nc")
v_era5 = xr.open_dataset(era5path+"e5.oper.an.ml.0_5_0_2_3_v.regn320uv.2021060100_2021060105.nc")
ps_era5 = xr.open_dataset(era5path+"e5.oper.an.ml.128_134_sp.regn320sc.2021060100_2021060105.nc")
phis_era5 = xr.open_dataset("./phis/ERA5_phis.nc")
phis_era5 = phis_era5.rename({'g4_lon_3':'longitude','g4_lat_2':'latitude'})
#phis_era5 = phis_era5.isel(initial_time0_hours=0, forecast_time1=0)
a_half = t_era5.a_half
b_half = t_era5.b_half
a_full = t_era5.a_model
b_full = t_era5.b_model

### Read in the CESM grid

In [5]:
cesm_gridh = xr.open_dataset("./grids/CAM6_horizontal.nc")
grid_out = xr.Dataset({'lat': (['lat'], cesm_gridh.lat.values)},{'lon':(['lon'], cesm_gridh.lon.values)})
cesm_gridv = xr.open_dataset("./grids/CAM6_vertical.nc")

### Using conservative regridding

In [6]:
reusewgt=False
wgtfile=outpath+"wgtfile.nc"

cesm_gridh = xr.open_dataset("./grids/CAM6_horizontal.nc")
grid_out = xr.Dataset({'lat': (['lat'], cesm_gridh.lat.values)},{'lon':(['lon'], cesm_gridh.lon.values)})

regridder = xe.Regridder(t_era5, grid_out, 'conservative', periodic=True, reuse_weights=reusewgt, filename=wgtfile)

phis_era5_rg = regridder(phis_era5.Z_GDS4_SFC)
t_era5_rg = regridder(t_era5.T)
u_era5_rg = regridder(u_era5.U)
v_era5_rg = regridder(v_era5.V)
ps_era5_rg = regridder(ps_era5.SP)

/glade/u/home/islas/miniconda3/envs/era5interpenv/lib/python3.10/site-packages/xesmf/backend.py:56: UserWarning: Latitude is outside of [-90, 90]
  warnings.warn('Latitude is outside of [-90, 90]')


### Reading in the CAM surface geopotential and calculating $\Delta$z and $\Delta \Phi_{s}$

In [8]:
phis_cam6 = xr.open_dataset("./phis/CAM6_phis.nc").isel(time=0)
deltaz = (phis_cam6.PHIS - phis_era5_rg.values)/9.81
deltaphis = (phis_era5_rg.values - phis_cam6.PHIS)

### Adjusting surface pressure

In [9]:
pfull = a_full + b_full*ps_era5_rg ; pfull = pfull.transpose("time","level","lat","lon")
phalf = a_half + b_half*ps_era5_rg ; phalf = phalf.transpose("time","half_level","lat","lon")
dp = phalf.isel(half_level=slice(1,phalf.half_level.size)).values - phalf.isel(half_level=slice(0,phalf.half_level.size-1))
dp = dp.rename(half_level='level')
dp = dp.transpose("time","level","lat","lon")
halfdpbelow = phalf.isel(half_level=slice(1,phalf.half_level.size)).values - pfull
halfdpbelow = halfdpbelow.transpose("time","level","lat","lon")

### Calculate the height of each level

In [10]:
#----Note, I haven't done exactly what's in the instructions here.  I used the dp between the interface below and the level for the
#---- k level
z = t_era5_rg.copy(deep=True)*0
CHT = (1./pfull)*dp*t_era5_rg
CHTk = (1./pfull)*halfdpbelow*t_era5_rg
for k in np.arange(0,pfull.level.size,1):
    z[:,k,:,:] = (R/g)*( CHT.isel(level=slice(k+1,CHT.level.size)).sum("level") + 
                                           CHTk.isel(level=k) )

### Find the temperature and the pressure at the first level above 150m

In [11]:
# Setting t_era5_rg where the height is below 150 to NaN's
t_era5_rg_forz = t_era5_rg.copy(deep=True)
p_era5_rg_forz = pfull.copy(deep=True)

t_era5_rg_forz = t_era5_rg_forz.where( z > 150, nan)
p_era5_rg_forz = p_era5_rg_forz.where( z > 150, nan)

# Find the minimum heights above 150
zabove150 = z.where( z > 150, nan) 
minz = zabove150.min("level")

# Grab out the T and p at that level
t_era5_rg_forz = t_era5_rg_forz.where( z == minz, nan)
p_era5_rg_forz = p_era5_rg_forz.where( z == minz, nan)

t_era5_rg_forz.transpose("time","lat","lon","level")
p_era5_rg_forz.transpose("time","lat","lon","level")

t_era5_rg_forz_stacked = t_era5_rg_forz.stack(z=("time","lat","lon","level"))
p_era5_rg_forz_stacked = p_era5_rg_forz.stack(z=("time","lat","lon","level"))

t_era5_rg_forz_stacked = t_era5_rg_forz_stacked.dropna("z")
p_era5_rg_forz_stacked = p_era5_rg_forz_stacked.dropna("z")

t_era5_150 = np.reshape(np.array(t_era5_rg_forz_stacked),[t_era5_rg_forz.time.size, t_era5_rg_forz.lat.size, t_era5_rg_forz.lon.size])
p_era5_150 = np.reshape(np.array(p_era5_rg_forz_stacked),[p_era5_rg_forz.time.size, p_era5_rg_forz.lat.size, p_era5_rg_forz.lon.size])

t_era5_150 = xr.DataArray(t_era5_150, dims=['time','lat','lon'],
                          coords=[t_era5_rg_forz.time, t_era5_rg_forz.lat, t_era5_rg_forz.lon], name='t_era5_150')
p_era5_150 = xr.DataArray(p_era5_150, dims=['time','lat','lon'],
                          coords=[p_era5_rg_forz.time, p_era5_rg_forz.lat, p_era5_rg_forz.lon], name='p_era5_150')

### Calculate the Tsurf and T$_{o}$

In [12]:
tsurf = t_era5_150 - dtdzst*(R/g)*( (ps_era5_rg/p_era5_150) - 1)*t_era5_150
to = tsurf - dtdzst*phis_era5_rg/g

### Calculate the modified $\Gamma$ and $T_{surf}$ for the extrapolation

In [13]:
tsurf_mod = tsurf.copy(deep=True)
gamma = tsurf.copy(deep=True)*0 + dtdzst

#--Mods for if T_{o} > 290.5 and T_{surf} <= 290.5
gamma = gamma.where( ~( (to > 290.5) & (tsurf <= 290.5) ), (290.5 - tsurf)*(g/phis_era5_rg) )

#--Mods for if T_{o} > 290.5 and T_{surf} > 290.5
gamma = gamma.where( ~( (to > 290.5) & (tsurf > 290.5)), 0)
tsurf_mod = tsurf_mod.where( ~( (to > 290.5) & (tsurf > 290.5)), 0.5*(290.5 + tsurf) )

#--Mods for if T_{surf} < 255
gamma = gamma.where( tsurf >= 255, dtdzst)
tsurf_mod = tsurf_mod.where( tsurf >= 255, 0.5*(255 + tsurf) )

### Extrapolate $p_{s}$ from $\Phi_{s}^{i}$ to $\Phi_{s}^{n}$ 

In [14]:
x = (np.array(gamma)*np.array(deltaphis) / (g*np.array(tsurf_mod))) 
psuse = ps_era5_rg*np.exp( (np.array(deltaphis)/(R*np.array(tsurf_mod)))*( 1 - (x/2.) + (x**2)/3) ) 
#!!!! modify this for the grid points where the deltaphis is really small.

### Perform the vertical interpolation

### !!! I think this needs to be better.  It's not going to be efficient.

In [15]:
newlevels = cesm_gridv.hyam*1e5 + cesm_gridv.hybm*psuse

In [16]:
### !!! Change to vertical

In [17]:
t_era5_rg = t_era5_rg.rename(level='lev')
a_full = a_full.rename(level='lev')
b_full = b_full.rename(level='lev')

In [18]:
ttest = t_era5_rg.isel(lon=0, lat=0, time=0)
pstest = psuse.isel(lon=0, lat=0, time=0)
tbottest = ttest.isel(lev=ttest.lev.size-1)
newlevtest = newlevels.isel(lon=0, lat=0, time=0)

In [19]:
tinterp = gc.interpolation.interp_hybrid_to_pressure(
    ttest, pstest, a_full, b_full, p0=1, new_levels=newlevtest.values, method='log', lev_dim='lev', extrapolate=False)

In [30]:
ttest = ttest.rename('T')
ttest.to_netcdf('dattest.nc')
pstest = pstest.rename('PS')
pstest.to_netcdf('dattest.nc', mode='a')
a_full.to_netcdf('dattest.nc', mode='a')
b_full.to_netcdf('dattest.nc', mode='a')
#newlevtest.to_netcdf('dattest.nc', mode='a')
tbottest = tbottest.rename('TBOT')
tbottest.to_netcdf('dattest.nc', mode='a')
phis_era5_rg.to_netcdf('dattest_phis.nc')

newlevtest.to_netcdf('dattest_newlevs.nc')


In [27]:
print(ttest)

<xarray.DataArray (lev: 137)>
array([186.20729, 196.09845, 201.5217 , 204.12112, 205.77226, 208.27309,
       210.52003, 210.23029, 206.49706, 205.93336, 205.31563, 203.45363,
       199.76317, 195.95024, 191.8652 , 188.60718, 185.49236, 182.84442,
       180.89369, 178.10297, 175.15602, 171.95988, 169.7062 , 168.56342,
       167.10811, 165.25577, 164.1448 , 163.694  , 163.12439, 162.08728,
       161.0049 , 160.22981, 159.52556, 158.6651 , 157.92522, 157.5447 ,
       157.52187, 157.63573, 157.73627, 157.84383, 158.02623, 158.21857,
       158.3134 , 158.36095, 158.49884, 158.79533, 159.20567, 159.66893,
       160.18497, 160.80406, 161.47388, 162.11266, 162.72287, 163.39293,
       164.12976, 164.85368, 165.53552, 166.23637, 167.03394, 167.9108 ,
       168.74278, 169.54945, 170.37967, 171.23306, 172.02733, 172.74413,
       173.41304, 174.0445 , 174.63358, 175.21278, 175.81917, 176.31897,
       176.65353, 176.90688, 177.08476, 177.16014, 177.17891, 177.19952,
       177.18939, 177

In [20]:
tinterp = gc.interpolation.interp_hybrid_to_pressure(
    ttest, pstest, a_full, b_full, p0=1, new_levels=newlevtest.values, method='log', lev_dim='lev', extrapolate=True,
    variable='temperature', t_bot=tbottest, phi_sfc=phis_era5_rg)

KeyError: "DataArray.cf does not understand the key 'vertical'. Use 'repr(DataArray.cf)' (or 'DataArray.cf' in a Jupyter environment) to see a list of key names that can be interpreted."

In [46]:
#tinterp = gc.interpolation.interp_hybrid_to_pressure(
#    t_era5_rg.isel(lon=0, lat=0, time=0),psuse.isel(lon=0, lat=0, time=0),a_full, b_full,p0=1, new_levels=newlevels.isel(lon=0, lat=0, time=0).values, lev_dim='level', method='log',
#    extrapolate=True, variable='temperature', t_bot=t_era5_rg.isel(level=t_era5_rg.level.size-1), phi_sfc=phis_era5_rg)
tinterp = gc.interpolation.interp_hybrid_to_pressure(
    t_era5_rg.isel(lon=0, lat=0, time=0),psuse.isel(lon=0, lat=0, time=0).values,a_full, b_full,p0=1, method='log',
    extrapolate=True, variable='temperature', lev_dim='lev',t_bot=t_era5_rg.isel(lev=t_era5_rg.lev.size-1), phi_sfc=phis_era5_rg)

KeyError: "DataArray.cf does not understand the key 'vertical'. Use 'repr(DataArray.cf)' (or 'DataArray.cf' in a Jupyter environment) to see a list of key names that can be interpreted."

In [38]:
print(psuse)

<xarray.DataArray (time: 6, lat: 192, lon: 288)>
array([[[ 52855.95719861,  52854.05032347,  52852.54098608, ...,
          52856.16433421,  52856.04071413,  52855.93759408],
        [ 68671.82100445,  68639.78447   ,  68604.15579284, ...,
          68759.945051  ,  68730.37984523,  68700.13183765],
        [ 69482.41207788,  69415.94325676,  69348.52650377, ...,
          69667.07750977,  69603.17950884,  69541.65004142],
        ...,
        [101455.19336064, 101458.27116274, 101461.34320858, ...,
         101441.38058314, 101446.10508067, 101450.7344476 ],
        [101454.16786155, 101454.75398203, 101455.40113251, ...,
         101452.59640279, 101453.20220572, 101453.61762034],
        [ 85169.54673612,  85170.0009438 ,  85170.44255618, ...,
          85169.40135325,  85169.30712152,  85169.38749533]],

       [[ 52859.49347336,  52857.56932619,  52856.10003892, ...,
          52859.8942306 ,  52859.54963368,  52859.48761432],
        [ 68661.74362004,  68629.78538267,  68594.2912

In [37]:
print(b_full)

<xarray.DataArray 'b_model' (level: 137)>
array([0.000000e+00, 0.000000e+00, 0.000000e+00, 0.000000e+00, 0.000000e+00,
       0.000000e+00, 0.000000e+00, 0.000000e+00, 0.000000e+00, 0.000000e+00,
       0.000000e+00, 0.000000e+00, 0.000000e+00, 0.000000e+00, 0.000000e+00,
       0.000000e+00, 0.000000e+00, 0.000000e+00, 0.000000e+00, 0.000000e+00,
       0.000000e+00, 0.000000e+00, 0.000000e+00, 0.000000e+00, 0.000000e+00,
       0.000000e+00, 0.000000e+00, 0.000000e+00, 0.000000e+00, 0.000000e+00,
       0.000000e+00, 0.000000e+00, 0.000000e+00, 0.000000e+00, 0.000000e+00,
       0.000000e+00, 0.000000e+00, 0.000000e+00, 0.000000e+00, 0.000000e+00,
       0.000000e+00, 0.000000e+00, 0.000000e+00, 0.000000e+00, 0.000000e+00,
       0.000000e+00, 0.000000e+00, 0.000000e+00, 0.000000e+00, 0.000000e+00,
       0.000000e+00, 0.000000e+00, 0.000000e+00, 0.000000e+00, 3.500000e-06,
       1.550000e-05, 4.150000e-05, 8.550000e-05, 1.555000e-04, 2.695000e-04,
       4.510000e-04, 7.260000e-04,

In [27]:
tinterp = gc.interpolation.interp_hybrid_to_pressure(
    t_era5_rg.isel(lon=0, lat=0, time=0),psuse.isel(lon=0, lat=0, time=0),a_full, b_full,p0=1, new_levels=newlevels.isel(lon=0, lat=0, time=0), lev_dim='level', method='log')

ValueError: cannot add coordinates with new dimensions to a DataArray